In [ ]:
!pip install requests pandas beautifulsoup4 tqdm nltk sentence-transformers numpy pinecone matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
!pip uninstall pinecone-client -y
!pip install pinecone

Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
  Using cached pinecone-7.0.2-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.0.2-py3-none-any.whl (516 kB)


In [51]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import re
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
import pinecone
import matplotlib.pyplot as plt
import time

In [52]:
from pinecone import Pinecone, ServerlessSpec

In [53]:
GROQ_API_KEY = "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"
client = Groq(api_key=GROQ_API_KEY)

In [54]:
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading punkt tokenizer...")
    nltk.download('punkt', quiet=True)
    print("Punkt tokenizer downloaded.")

# Assignment 2

In [ ]:
def collect_data():
    """
    Collect data from Wikipedia and custom sources in both English and Urdu
    """
    print("\n=== Step 3: Collecting Data ===")

    # English articles to collect
    en_articles = [
        'Nutrition',
        'Healthy_diet',
        'Pakistani_cuisine',
        'Indian_cuisine',
        'South_Asian_cuisine',
        'Halal',
        'Desi_cuisine'
    ]

    # Urdu articles to collect (reduced list to more likely existing articles)
    ur_articles = [
        'Pakistani_cuisine',
        'Indian_cuisine'
    ]

    def get_wikipedia_content(title, lang='en', max_retries=3, delay=2):
        for attempt in range(max_retries):
            try:
                url = f'https://{lang}.wikipedia.org/w/api.php'
                params = {
                    'action': 'query',
                    'format': 'json',
                    'titles': title,
                    'prop': 'extracts',
                    'exintro': True,
                    'explaintext': True
                }
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                }
                response = requests.get(url, params=params, headers=headers)
                response.raise_for_status()

                try:
                    data = response.json()
                except json.JSONDecodeError:
                    print(f"Error: Invalid JSON response for {title} in {lang}")
                    if attempt < max_retries - 1:
                        time.sleep(delay)
                        continue
                    return None

                pages = data['query']['pages']
                page = next(iter(pages.values()))

                if 'extract' not in page:
                    print(f"Warning: No content found for {title} in {lang}")
                    return None

                return page['extract']

            except requests.exceptions.RequestException as e:
                print(f"Error fetching {title} in {lang}: {str(e)}")
                if attempt < max_retries - 1:
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                    continue
                return None
            except Exception as e:
                print(f"Unexpected error for {title} in {lang}: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(delay)
                    continue
                return None

    # Additional Urdu food and nutrition data
    urdu_food_data = [
        {
            'title': 'Pakistani Food',
            'content': """
            پاکستانی کھانوں میں بریانی، کڑاہی، نہاری، چپلی کباب، سموسے، پکوڑے، حلیم،
            اور مختلف قسم کی روٹیاں شامل ہیں۔ یہ کھانے صحت بخش اجزاء سے بنائے جاتے ہیں
            اور متوازن غذا کا حصہ ہیں۔
            """,
            'language': 'ur'
        },
        {
            'title': 'Healthy Eating',
            'content': """
            صحت مند کھانے میں تازہ سبزیاں، پھل، دالیں، اور کم چکنائی والی پروٹین شامل ہونی چاہئیں۔
            روزانہ کم از کم 8 گلاس پانی پینا چاہیے اور پروسیسڈ فوڈز سے پرہیز کرنا چاہیے۔
            """,
            'language': 'ur'
        },
        {
            'title': 'Traditional Pakistani Diet',
            'content': """
            روایتی پاکستانی غذا میں چاول، روٹی، دالیں، سبزیاں، اور گوشت شامل ہیں۔
            یہ غذا توانائی اور غذائیت سے بھرپور ہوتی ہے۔
            """,
            'language': 'ur'
        },
        {
            'title': 'Nutritional Benefits',
            'content': """
            پاکستانی کھانوں میں موجود اجزاء صحت کے لیے بہت فائدہ مند ہیں۔
            دالیں پروٹین کا اچھا ذریعہ ہیں، سبزیاں وٹامنز اور معدنیات سے بھرپور ہیں،
            اور گوشت آئرن اور پروٹین فراہم کرتا ہے۔
            """,
            'language': 'ur'
        },
        {
            'title': 'Balanced Diet',
            'content': """
            متوازن غذا میں کاربوہائیڈریٹس، پروٹین، چکنائی، وٹامنز اور معدنیات کی مناسب مقدار ہونی چاہیے۔
            پاکستانی کھانوں میں یہ تمام اجزاء موجود ہیں، لیکن اعتدال میں کھانا ضروری ہے۔
            """,
            'language': 'ur'
        },
        {
            'title': 'Cooking Methods',
            'content': """
            پاکستانی کھانوں میں مختلف طریقے استعمال ہوتے ہیں جیسے بھوننا، پکانا، اور تڑکا لگانا۔
            یہ طریقے کھانوں کی غذائیت کو برقرار رکھتے ہیں اور ذائقہ بڑھاتے ہیں۔
            """,
            'language': 'ur'
        }
    ]

    # Collect English Wikipedia articles
    en_data = []
    for article in tqdm(en_articles, desc='Collecting English articles'):
        content = get_wikipedia_content(article, 'en')
        if content:  # Only add if content was successfully retrieved
            en_data.append({
                'title': article,
                'content': content,
                'language': 'en'
            })

    # Collect Urdu content
    ur_data = []
    for article in tqdm(ur_articles, desc='Collecting Urdu articles'):
        content = get_wikipedia_content(article, 'ur')
        if content:  # Only add if content was successfully retrieved
            ur_data.append({
                'title': article,
                'content': content,
                'language': 'ur'
            })

    # Add additional Urdu food data
    ur_data.extend(urdu_food_data)

    # Combine all data
    all_data = en_data + ur_data
    df = pd.DataFrame(all_data)
    print(f"\nData Collection Summary:")
    print(f"Total documents collected: {len(df)}")
    print(f"English documents: {len(en_data)}")
    print(f"Urdu documents: {len(ur_data)}")

    return df


In [ ]:
def preprocess_text(df):
    """
    Clean and preprocess the collected text while preserving Urdu characters
    """
    print("\n=== Step 4: Preprocessing Text ===")

    def clean_text(text):
        # Keep Urdu characters and punctuation
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s.,!?؟،-]', '', text)  # Modified to keep Urdu punctuation
        return text.strip()

    df['cleaned_content'] = df['content'].apply(clean_text)
    print("Sample of cleaned text (English):")
    print(df[df['language'] == 'en']['cleaned_content'].iloc[0][:200])
    print("\nSample of cleaned text (Urdu):")
    print(df[df['language'] == 'ur']['cleaned_content'].iloc[0][:200])

    return df

In [ ]:
def apply_chunking(df):
    """
    Implement both fixed-length and sentence-based chunking strategies
    """
    print("\n=== Step 5: Applying Chunking Strategies ===")

    def create_fixed_length_chunks(text, chunk_size=500, overlap=50):
        chunks = []
        start = 0
        text_length = len(text)

        while start < text_length:
            end = start + chunk_size
            chunk = text[start:end]
            chunks.append(chunk)
            start = end - overlap

        return chunks

    def create_sentence_chunks(text, max_chunk_size=500):
        # Simple sentence splitting for both English and Urdu
        # For English, use NLTK's sent_tokenize
        # For Urdu, split on common sentence endings
        if any('\u0600' <= char <= '\u06FF' for char in text):  # Check if text contains Urdu characters
            # Split Urdu text on common sentence endings
            sentences = re.split(r'[.!?؟،]', text)
            sentences = [s.strip() for s in sentences if s.strip()]
        else:
            # Use NLTK for English text
            try:
                sentences = sent_tokenize(text)
            except:
                # Fallback to simple splitting if NLTK fails
                sentences = re.split(r'[.!?]', text)
                sentences = [s.strip() for s in sentences if s.strip()]

        chunks = []
        current_chunk = []
        current_size = 0

        for sentence in sentences:
            sentence_size = len(sentence)
            if current_size + sentence_size > max_chunk_size and current_chunk:
                chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
                current_size = sentence_size
            else:
                current_chunk.append(sentence)
                current_size += sentence_size

        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    # Apply both chunking strategies
    df['fixed_chunks'] = df['cleaned_content'].apply(lambda x: create_fixed_length_chunks(x))
    df['sentence_chunks'] = df['cleaned_content'].apply(lambda x: create_sentence_chunks(x))

    print("Sample chunks from both strategies (English):")
    print("\nFixed-length chunks:")
    print(df[df['language'] == 'en']['fixed_chunks'].iloc[0][0])
    print("\nSentence-based chunks:")
    print(df[df['language'] == 'en']['sentence_chunks'].iloc[0][0])

    return df

In [ ]:
def generate_embeddings(df):
    """
    Generate embeddings using multilingual models that support both English and Urdu
    """
    print("\n=== Step 6: Generating Embeddings ===")

    def get_embeddings(texts, model_name, batch_size=32):
        print(f"Loading model: {model_name}")
        model = SentenceTransformer(model_name)

        # Process in batches
        all_embeddings = []
        for i in tqdm(range(0, len(texts), batch_size), desc=f"Generating embeddings with {model_name}"):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = model.encode(batch_texts, show_progress_bar=False)
            all_embeddings.extend(batch_embeddings)

        return np.array(all_embeddings)

    # Using a single best model for both languages
    model_name = 'sentence-transformers/LaBSE'  # Language-agnostic BERT Sentence Embedding

    # Process only sentence chunks as they're more meaningful
    print("Processing sentence chunks...")
    all_chunks = []
    for chunks in df['sentence_chunks']:
        all_chunks.extend(chunks)

    print(f"Total chunks to process: {len(all_chunks)}")

    # Generate embeddings
    embeddings = get_embeddings(all_chunks, model_name)

    # Save embeddings
    embeddings_dict = {'sentence_chunks_LaBSE': embeddings}
    np.save('embeddings_sentence_chunks_LaBSE.npy', embeddings)

    print("Embeddings generation completed!")
    return embeddings_dict


In [ ]:
pip install --upgrade pinecone-client

  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
Using cached pinecone_client-6.0.0-py3-none-any.whl (6.7 kB)


In [ ]:
def setup_pinecone():
    """
    Connect to the existing Pinecone index 'urdu-eng-rag' in AWS us-east-1 with dimension 768.
    """
    print("\n=== Step 7: Setting up Pinecone ===")
    try:
        PINECONE_API_KEY = "pcsk_ffw9K_3f5Semen1JhXkdPBYdJsZEWmZfCP7T1tsRixFSHb1GGuWiL7ij21K24rsEhTbBo"
        index_name = "urdu-eng-rag"
        dimension = 768  # Must match your index

        pc = Pinecone(api_key=PINECONE_API_KEY)

        # Only create if not exists (should already exist as per your screenshot)
        if index_name not in pc.list_indexes().names():
            print(f"Creating new index: {index_name}")
            pc.create_index(
                name=index_name,
                dimension=dimension,
                metric='cosine',
                spec=ServerlessSpec(
                    cloud='aws',
                    region='us-east-1'
                )
            )
            print("Index created successfully!")
        else:
            print(f"Using existing index: {index_name}")

        index = pc.Index(index_name)
        print("Pinecone setup completed successfully!")
        return index

    except Exception as e:
        print(f"Error setting up Pinecone: {str(e)}")
        raise

In [ ]:
def store_embeddings(index, df, embeddings_dict):
    """
    Store the generated embeddings in Pinecone with appropriate metadata
    """
    print("\n=== Step 8: Storing Embeddings ===")

    def store_embeddings_batch(embeddings, chunks, metadata, batch_size=100):
        for i in range(0, len(embeddings), batch_size):
            batch_embeddings = embeddings[i:i + batch_size]
            batch_chunks = chunks[i:i + batch_size]

            vectors = []
            for j, (embedding, chunk) in enumerate(zip(batch_embeddings, batch_chunks)):
                vector = {
                    'id': f'vec_{i+j}',
                    'values': embedding.tolist(),
                    'metadata': {
                        'text': chunk,
                        **metadata
                    }
                }
                vectors.append(vector)

            index.upsert(vectors=vectors)

    # Get the embeddings and chunks
    embeddings = embeddings_dict['sentence_chunks_LaBSE']
    all_chunks = []
    for chunks in df['sentence_chunks']:
        all_chunks.extend(chunks)

    print(f"Storing {len(all_chunks)} chunks with their embeddings...")

    # Store embeddings with metadata
    metadata = {
        'chunking_strategy': 'sentence_chunks',
        'model': 'LaBSE'
    }

    store_embeddings_batch(embeddings, all_chunks, metadata)
    print("Embeddings stored successfully!")

In [ ]:
def query_pinecone(index, query_text, model_name='sentence-transformers/LaBSE', top_k=5):
    model = SentenceTransformer(model_name)
    query_embedding = model.encode(query_text)
    results = index.query(
        vector=query_embedding.tolist(),
        top_k=top_k,
        include_metadata=True
    )
    return results

In [ ]:
def test_rag(index):
    """
    Test the chatbot with queries in both English and Urdu
    """
    print("\n=== Step 9: Testing RAG System ===")

    test_queries = [
        "What are the health benefits of Pakistani cuisine?",
        "پاکستانی کھانوں کے صحت کے فوائد کیا ہیں؟",
        "How to maintain a balanced diet with traditional Pakistani food?",
        "روایتی پاکستانی کھانوں کے ساتھ متوازن غذا کیسے برقرار رکھیں؟"
    ]

    for query in test_queries:
        print(f"\nQuery: {query}")
        results = query_pinecone(index, query)

        print("\nTop results:")
        for match in results.matches:
            print(f"\nScore: {match.score}")
            print(f"Text: {match.metadata['text'][:200]}...")
            print(f"Chunking Strategy: {match.metadata['chunking_strategy']}")
            print(f"Model: {match.metadata['model']}")

In [ ]:
def analyze_performance(index):
    """
    Analyze the performance of different models and chunking strategies
    """
    print("\n=== Step 10: Analyzing Performance ===")

    def evaluate_combinations(queries):
        results = []
        models = [
            'sentence-transformers/LaBSE'
        ]
        chunking_strategies = ['sentence_chunks']
        for query in queries:
            for model in models:
                for strategy in chunking_strategies:
                    results_query = query_pinecone(index, query, model)
                    avg_score = sum(match.score for match in results_query.matches) / len(results_query.matches)
                    results.append({
                        'query': query,
                        'model': model.split('/')[-1],
                        'strategy': strategy,
                        'avg_score': avg_score
                    })
        return pd.DataFrame(results)

    test_queries = [
        "What are the health benefits of Pakistani cuisine?",
        "پاکستانی کھانوں کے صحت کے فوائد کیا ہیں؟"
    ]

    results_df = evaluate_combinations(test_queries)

    # Create pivot table for visualization
    pivot_df = results_df.pivot_table(
        values='avg_score',
        index='model',
        columns='strategy'
    )

    # Plot results
    plt.figure(figsize=(10, 6))
    pivot_df.plot(kind='bar')
    plt.title('Model-Chunking Strategy Performance Comparison')
    plt.xlabel('Model')
    plt.ylabel('Average Score')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('performance_comparison.png')
    plt.close()

    # Print detailed results
    print("\nDetailed Results:")
    print(results_df.sort_values('avg_score', ascending=False))

    return results_df

In [ ]:
def main():
    try:
        # Step 1: Collect data
        df = collect_data()

        # Step 2: Preprocess text
        df = preprocess_text(df)

        # Step 3: Apply chunking
        df = apply_chunking(df)

        # Step 4: Generate embeddings
        embeddings_dict = generate_embeddings(df)

        # Step 5: Setup Pinecone
        index = setup_pinecone()

        # Step 6: Store embeddings
        store_embeddings(index, df, embeddings_dict)

        # Step 7: Test RAG
        test_rag(index)

        # Step 8: Analyze performance
        results_df = analyze_performance(index)

        print("\nAll steps completed successfully!")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


=== Step 3: Collecting Data ===



Data Collection Summary:
Total documents collected: 12
English documents: 6
Urdu documents: 6

=== Step 4: Preprocessing Text ===
Sample of cleaned text (English):
Nutrition is the biochemical and physiological process by which an organism uses food and water to support its life. The intake of these substances provides organisms with nutrients divided into macro

Sample of cleaned text (Urdu):
پاکستانی کھانوں میں بریانی، کڑاہی، نہاری، چپلی کباب، سموسے، پکوڑے، حلیم، اور مختلف قسم کی روٹیاں شامل ہیں یہ کھانے صحت بخش اجزاء سے بنائے جاتے ہیں اور متوازن غذا کا حصہ ہیں

=== Step 5: Applying Chunking Strategies ===
Sample chunks from both strategies (English):

Fixed-length chunks:
Nutrition is the biochemical and physiological process by which an organism uses food and water to support its life. The intake of these substances provides organisms with nutrients divided into macro- and micro- which can be metabolized to create energy and chemical structures too much or too little of an essenti


Generating embeddings with sentence-transformers/LaBSE: 100%|██████████| 1/1 [00:07<00:00,  7.26s/it]


Embeddings generation completed!

=== Step 7: Setting up Pinecone ===
Using existing index: urdu-eng-rag
Pinecone setup completed successfully!

=== Step 8: Storing Embeddings ===
Storing 28 chunks with their embeddings...
Embeddings stored successfully!

=== Step 9: Testing RAG System ===

Query: What are the health benefits of Pakistani cuisine?

Top results:

Score: 0.591887236
Text: پاکستانی کھانوں میں موجود اجزاء صحت کے لیے بہت فائدہ مند ہیں دالیں پروٹین کا اچھا ذریعہ ہیں سبزیاں وٹامنز اور معدنیات سے بھرپور ہیں اور گوشت آئرن اور پروٹین فراہم کرتا ہے...
Chunking Strategy: sentence_chunks
Model: LaBSE

Score: 0.572941363
Text: پاکستانی کھانوں میں بریانی کڑاہی نہاری چپلی کباب سموسے پکوڑے حلیم اور مختلف قسم کی روٹیاں شامل ہیں یہ کھانے صحت بخش اجزاء سے بنائے جاتے ہیں اور متوازن غذا کا حصہ ہیں...
Chunking Strategy: sentence_chunks
Model: LaBSE

Score: 0.55020833
Text: روایتی پاکستانی غذا میں چاول روٹی دالیں سبزیاں اور گوشت شامل ہیں یہ غذا توانائی اور غذائیت سے بھرپور ہوتی ہے...
Chunking

<Figure size 1000x600 with 0 Axes>

# Assignment 3

In [55]:
!pip install groq

In [56]:
from groq import Groq

In [57]:
import os

In [78]:
# Set Groq API key as environment variable
os.environ["GROQ_API_KEY"] = "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"

# Initialize Groq client
client = Groq()

# List available models
try:
    models = client.models.list()
    print("Available models:")
    print(models)  # This will print the raw response
except Exception as e:
    print(f"Error listing models: {str(e)}")

Available models:
ModelListResponse(data=[Model(id='compound-beta-mini', created=1742953279, object='model', owned_by='Groq', active=True, context_window=8192, public_apps=None, max_completion_tokens=8192), Model(id='playai-tts-arabic', created=1740682783, object='model', owned_by='PlayAI', active=True, context_window=8192, public_apps=None, max_completion_tokens=8192), Model(id='qwen-qwq-32b', created=1741214760, object='model', owned_by='Alibaba Cloud', active=True, context_window=131072, public_apps=None, max_completion_tokens=131072), Model(id='llama-3.1-8b-instant', created=1693721698, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=131072), Model(id='compound-beta', created=1740880017, object='model', owned_by='Groq', active=True, context_window=8192, public_apps=None, max_completion_tokens=8192), Model(id='llama3-70b-8192', created=1693721698, object='model', owned_by='Meta', active=True, context_window=8192, public_ap

In [81]:
# Set Groq API key as environment variable
os.environ["GROQ_API_KEY"] = "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"

# Initialize Groq client
client = Groq()

# Test the connection
try:
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  # Updated model name
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Hello, are you working?"}
        ],
        max_tokens=50,
        temperature=0.3
    )
    print("Groq connection successful!")
except Exception as e:
    print(f"Error connecting to Groq: {str(e)}")

Groq connection successful!


In [82]:
def collect_multilingual_data():
    """
    Enhanced data collection from multiple sources in English and Urdu
    """
    print("\n=== Step 1: Collecting Multilingual Data ===")

    # English sources
    en_sources = {
        'wikipedia': [
            'Nutrition',
            'Healthy_diet',
            'Pakistani_cuisine',
            'Indian_cuisine',
            'South_Asian_cuisine',
            'Halal',
            'Desi_cuisine',
            'Traditional_medicine',
            'Herbal_medicine',
            'Ayurveda'
        ],
        'custom': [
            {
                'title': 'Traditional Pakistani Cooking Methods',
                'content': """
                Traditional Pakistani cooking methods include:
                1. Dum cooking - slow cooking in sealed containers
                2. Tandoori cooking - clay oven cooking
                3. Bhunna - sautéing and browning
                4. Dhuanaar - smoking with charcoal
                These methods preserve nutrients and enhance flavors.
                """
            },
            {
                'title': 'Health Benefits of Spices',
                'content': """
                Common Pakistani spices and their health benefits:
                - Turmeric: Anti-inflammatory properties
                - Cumin: Aids digestion
                - Coriander: Rich in antioxidants
                - Cardamom: Helps with respiratory issues
                - Cinnamon: Regulates blood sugar
                """
            }
        ]
    }

    # Urdu sources
    ur_sources = {
        'wikipedia': [
            'Pakistani_cuisine',
            'Indian_cuisine',
            'Traditional_medicine'
        ],
        'custom': [
            {
                'title': 'پاکستانی کھانوں کی صحت بخش خصوصیات',
                'content': """
                پاکستانی کھانوں میں استعمال ہونے والے مصالحے صحت کے لیے بہت فائدہ مند ہیں۔
                ہلدی میں اینٹی انفلیمیٹری خصوصیات ہیں، زیرہ ہاضمہ کو بہتر بناتا ہے،
                دھنیا اینٹی آکسیڈنٹس سے بھرپور ہے، الائچی سانس کی بیماریوں میں مدد کرتی ہے،
                اور دارچینی خون میں شکر کی سطح کو کنٹرول کرتی ہے۔
                """
            },
            {
                'title': 'روایتی پاکستانی کھانوں کے طریقے',
                'content': """
                روایتی پاکستانی کھانوں کے طریقے:
                1. دم پخت - مہر بند برتنوں میں آہستہ پکانا
                2. تندوری - مٹی کے تندور میں پکانا
                3. بھوننا - ہلکی آنچ پر پکانا
                4. دھونیار - کوئلے کی دھونی دینا
                یہ طریقے غذائیت کو برقرار رکھتے ہیں اور ذائقہ بڑھاتے ہیں۔
                """
            }
        ]
    }

    def get_wikipedia_content(title, lang='en', max_retries=3, delay=2):
        for attempt in range(max_retries):
            try:
                url = f'https://{lang}.wikipedia.org/w/api.php'
                params = {
                    'action': 'query',
                    'format': 'json',
                    'titles': title,
                    'prop': 'extracts',
                    'exintro': True,
                    'explaintext': True
                }
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
                }
                response = requests.get(url, params=params, headers=headers)
                response.raise_for_status()

                data = response.json()
                pages = data['query']['pages']
                page = next(iter(pages.values()))

                if 'extract' not in page:
                    print(f"Warning: No content found for {title} in {lang}")
                    return None

                return page['extract']

            except Exception as e:
                print(f"Error fetching {title} in {lang}: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(delay)
                    continue
                return None

    # Collect data from all sources
    all_data = []

    # Collect English Wikipedia articles
    for article in tqdm(en_sources['wikipedia'], desc='Collecting English Wikipedia articles'):
        content = get_wikipedia_content(article, 'en')
        if content:
            all_data.append({
                'title': article,
                'content': content,
                'language': 'en',
                'source': 'wikipedia'
            })

    # Add English custom content
    for item in en_sources['custom']:
        all_data.append({
            'title': item['title'],
            'content': item['content'],
            'language': 'en',
            'source': 'custom'
        })

    # Collect Urdu Wikipedia articles
    for article in tqdm(ur_sources['wikipedia'], desc='Collecting Urdu Wikipedia articles'):
        content = get_wikipedia_content(article, 'ur')
        if content:
            all_data.append({
                'title': article,
                'content': content,
                'language': 'ur',
                'source': 'wikipedia'
            })

    # Add Urdu custom content
    for item in ur_sources['custom']:
        all_data.append({
            'title': item['title'],
            'content': item['content'],
            'language': 'ur',
            'source': 'custom'
        })

    # Convert to DataFrame
    df = pd.DataFrame(all_data)

    print("\nData Collection Summary:")
    print(f"Total documents: {len(df)}")
    print(f"English documents: {len(df[df['language'] == 'en'])}")
    print(f"Urdu documents: {len(df[df['language'] == 'ur'])}")
    print(f"Wikipedia sources: {len(df[df['source'] == 'wikipedia'])}")
    print(f"Custom sources: {len(df[df['source'] == 'custom'])}")

    return df

In [83]:
def preprocess_text(df):
    """
    Clean and preprocess the collected text while preserving Urdu characters
    """
    print("\n=== Step 2: Preprocessing Text ===")

    def clean_text(text):
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s.,!?؟،-]', '', text)
        return text.strip()

    df['cleaned_content'] = df['content'].apply(clean_text)
    print("Sample of cleaned text (English):")
    print(df[df['language'] == 'en']['cleaned_content'].iloc[0][:200])
    print("\nSample of cleaned text (Urdu):")
    print(df[df['language'] == 'ur']['cleaned_content'].iloc[0][:200])

    return df


In [84]:
def apply_enhanced_chunking(df):
    """
    Apply multiple chunking strategies with improved handling
    """
    print("\n=== Step 3: Applying Enhanced Chunking Strategies ===")

    def create_fixed_length_chunks(text, chunk_size=500, overlap=50):
        chunks = []
        start = 0
        text_length = len(text)

        while start < text_length:
            end = start + chunk_size
            chunk = text[start:end]
            chunks.append(chunk)
            start = end - overlap

        return chunks

    def create_sentence_chunks(text, max_chunk_size=500):
        if any('\u0600' <= char <= '\u06FF' for char in text):
            sentences = re.split(r'[.!?؟،]', text)
            sentences = [s.strip() for s in sentences if s.strip()]
        else:
            try:
                sentences = sent_tokenize(text)
            except:
                sentences = re.split(r'[.!?]', text)
                sentences = [s.strip() for s in sentences if s.strip()]

        chunks = []
        current_chunk = []
        current_size = 0

        for sentence in sentences:
            sentence_size = len(sentence)
            if current_size + sentence_size > max_chunk_size and current_chunk:
                chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
                current_size = sentence_size
            else:
                current_chunk.append(sentence)
                current_size += sentence_size

        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    def create_semantic_chunks(text, max_chunk_size=500):
        """
        Create chunks based on semantic units (paragraphs, sections)
        """
        paragraphs = text.split('\n\n')
        chunks = []
        current_chunk = []
        current_size = 0

        for para in paragraphs:
            para = para.strip()
            if not para:
                continue

            para_size = len(para)
            if current_size + para_size > max_chunk_size and current_chunk:
                chunks.append('\n\n'.join(current_chunk))
                current_chunk = [para]
                current_size = para_size
            else:
                current_chunk.append(para)
                current_size += para_size

        if current_chunk:
            chunks.append('\n\n'.join(current_chunk))

        return chunks

    # Apply all chunking strategies
    df['fixed_chunks'] = df['cleaned_content'].apply(lambda x: create_fixed_length_chunks(x))
    df['sentence_chunks'] = df['cleaned_content'].apply(lambda x: create_sentence_chunks(x))
    df['semantic_chunks'] = df['cleaned_content'].apply(lambda x: create_semantic_chunks(x))

    # Print chunking statistics
    print("\nChunking Statistics:")
    for strategy in ['fixed_chunks', 'sentence_chunks', 'semantic_chunks']:
        total_chunks = sum(len(chunks) for chunks in df[strategy])
        avg_chunks = total_chunks / len(df)
        print(f"{strategy}:")
        print(f"  Total chunks: {total_chunks}")
        print(f"  Average chunks per document: {avg_chunks:.2f}")

    return df

In [85]:
def generate_multilingual_embeddings(df):
    """
    Generate embeddings using LaBSE model
    """
    print("\n=== Step 4: Generating Multilingual Embeddings ===")

    def get_embeddings(texts, model_name, batch_size=32):
        print(f"Loading model: {model_name}")
        model = SentenceTransformer(model_name)

        # Verify model dimension
        test_embedding = model.encode("test")
        embedding_dim = len(test_embedding)
        print(f"Model {model_name} generates {embedding_dim}-dimensional embeddings")

        if embedding_dim != 768:
            raise ValueError(f"Model {model_name} generates {embedding_dim}-dimensional embeddings, but we need 768-dimensional embeddings")

        all_embeddings = []
        for i in tqdm(range(0, len(texts), batch_size), desc=f"Generating embeddings with {model_name}"):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = model.encode(batch_texts, show_progress_bar=False)
            all_embeddings.extend(batch_embeddings)

        return np.array(all_embeddings)

    # Using only LaBSE model for multilingual support
    model = 'sentence-transformers/LaBSE'  # Language-agnostic BERT (768d)

    chunking_strategies = ['fixed_chunks', 'sentence_chunks', 'semantic_chunks']
    embeddings_dict = {}

    for strategy in chunking_strategies:
        print(f"\nProcessing {strategy}...")
        all_chunks = []
        for chunks in df[strategy]:
            all_chunks.extend(chunks)

        print(f"Total chunks to process: {len(all_chunks)}")

        try:
            print(f"\nGenerating embeddings with {model}")
            embeddings = get_embeddings(all_chunks, model)

            # Verify embedding dimensions
            if embeddings.shape[1] != 768:
                print(f"Warning: {model} generated {embeddings.shape[1]}-dimensional embeddings, skipping...")
                continue

            key = f"{strategy}_LaBSE"
            embeddings_dict[key] = embeddings
            np.save(f'embeddings_{key}.npy', embeddings)
            print(f"Successfully generated and saved embeddings for {key}")

        except Exception as e:
            print(f"Error processing {model}: {str(e)}")
            raise

    if not embeddings_dict:
        raise ValueError("No embeddings were successfully generated. Please check the model configuration.")

    return embeddings_dict


In [86]:
def setup_pinecone():
    """
    Connect to Pinecone index
    """
    print("\n=== Step 5: Setting up Pinecone ===")
    try:
        # Replace with your Pinecone API key
        PINECONE_API_KEY = "pcsk_ffw9K_3f5Semen1JhXkdPBYdJsZEWmZfCP7T1tsRixFSHb1GGuWiL7ij21K24rsEhTbBo"
        index_name = "urdu-eng-rag"
        dimension = 768

        pc = Pinecone(api_key=PINECONE_API_KEY)

        if index_name not in pc.list_indexes().names():
            print(f"Creating new index: {index_name}")
            pc.create_index(
                name=index_name,
                dimension=dimension,
                metric='cosine',
                spec=ServerlessSpec(
                    cloud='aws',
                    region='us-east-1'
                )
            )
            print("Index created successfully!")
        else:
            print(f"Using existing index: {index_name}")

        index = pc.Index(index_name)
        print("Pinecone setup completed successfully!")
        return index

    except Exception as e:
        print(f"Error setting up Pinecone: {str(e)}")
        raise

In [87]:
def store_enhanced_embeddings(index, df, embeddings_dict):
    """
    Store embeddings with enhanced metadata
    """
    print("\n=== Step 6: Storing Enhanced Embeddings ===")

    def store_embeddings_batch(embeddings, chunks, metadata, batch_size=100):
        # Verify embedding dimensions before storing
        if embeddings.shape[1] != 768:
            raise ValueError(f"Embedding dimension {embeddings.shape[1]} does not match required dimension 768")

        for i in range(0, len(embeddings), batch_size):
            batch_embeddings = embeddings[i:i + batch_size]
            batch_chunks = chunks[i:i + batch_size]

            vectors = []
            for j, (embedding, chunk) in enumerate(zip(batch_embeddings, batch_chunks)):
                vector = {
                    'id': f'vec_{i+j}',
                    'values': embedding.tolist(),
                    'metadata': {
                        'text': chunk,
                        **metadata
                    }
                }
                vectors.append(vector)

            try:
                index.upsert(vectors=vectors)
                print(f"Successfully stored batch {i//batch_size + 1}")
            except Exception as e:
                print(f"Error storing batch {i//batch_size + 1}: {str(e)}")
                raise

    for strategy in ['fixed_chunks', 'sentence_chunks', 'semantic_chunks']:
        print(f"\nProcessing {strategy}...")
        all_chunks = []
        for chunks in df[strategy]:
            all_chunks.extend(chunks)

        key = f"{strategy}_LaBSE"
        if key in embeddings_dict:
            try:
                print(f"Storing embeddings for {key}")
                embeddings = embeddings_dict[key]

                # Verify embedding dimensions
                if embeddings.shape[1] != 768:
                    print(f"Warning: {key} has incorrect dimension {embeddings.shape[1]}, skipping...")
                    continue

                metadata = {
                    'chunking_strategy': strategy,
                    'model': 'LaBSE',
                    'language': 'multilingual'
                }

                store_embeddings_batch(embeddings, all_chunks, metadata)
                print(f"Successfully stored all embeddings for {key}")

            except Exception as e:
                print(f"Error storing embeddings for {key}: {str(e)}")
                raise

    print("All embeddings stored successfully!")

In [88]:
def summarize_context(context_chunks, max_chunk_size=1000):
   try:
        summaries = []
        for chunk in context_chunks:
            if len(chunk) > max_chunk_size:
                sub_chunks = [chunk[i:i+max_chunk_size] for i in range(0, len(chunk), max_chunk_size)]
                chunk_summaries = []
                for sub_chunk in sub_chunks:
                    prompt = f"""Summarize the following text, focusing on key information and main points:
                    {sub_chunk}
                    Summary:"""

                    response = client.chat.completions.create(
                        model="llama-3.3-70b-versatile",  # Using Mixtral model from Groq
                        messages=[
                            {"role": "system", "content": "You are a helpful summarizer."},
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=150,
                        temperature=0.3
                    )
                    chunk_summaries.append(response.choices[0].message.content.strip())

                summaries.append(" ".join(chunk_summaries))
            else:
                summaries.append(chunk)

        return summaries
   except Exception as e:
        print(f"Error in summarize_context: {str(e)}")
        return context_chunks

In [89]:
def rank_context_chunks(chunks, query, model='sentence-transformers/LaBSE'):
    """
    Rank context chunks by relevance to the query
    """
    model = SentenceTransformer(model)
    query_embedding = model.encode(query)
    chunk_embeddings = model.encode(chunks)

    similarities = np.dot(chunk_embeddings, query_embedding) / (
        np.linalg.norm(chunk_embeddings, axis=1) * np.linalg.norm(query_embedding)
    )

    ranked_chunks = [(chunk, score) for chunk, score in zip(chunks, similarities)]
    ranked_chunks.sort(key=lambda x: x[1], reverse=True)

    return ranked_chunks

In [90]:
def filter_redundant_chunks(chunks, similarity_threshold=0.85):
    """
    Remove redundant chunks based on similarity
    """
    model = SentenceTransformer('sentence-transformers/LaBSE')
    embeddings = model.encode(chunks)

    unique_chunks = []
    for i, chunk in enumerate(chunks):
        is_redundant = False
        for unique_chunk in unique_chunks:
            similarity = np.dot(embeddings[i], model.encode(unique_chunk)) / (
                np.linalg.norm(embeddings[i]) * np.linalg.norm(model.encode(unique_chunk))
            )
            if similarity > similarity_threshold:
                is_redundant = True
                break
        if not is_redundant:
            unique_chunks.append(chunk)

    return unique_chunks

In [91]:
def build_enhanced_prompt(context, query, system_instruction=None):
    """
    Build an enhanced prompt with better structure and instructions
    """
    if system_instruction is None:
        system_instruction = """You are a helpful assistant that provides accurate and relevant information.
        Use the provided context to answer the question. If the answer is not in the context, say so.
        Ensure your response is:
        1. Accurate and based on the context
        2. Well-structured and easy to understand
        3. Comprehensive but concise
        4. Relevant to the specific question asked"""

    prompt = f"""{system_instruction}

Context:
{context}

Question: {query}

Please provide a detailed answer based on the context above. If the context doesn't contain enough information to answer the question, please state that clearly."""

    return prompt

In [92]:
def evaluate_response_quality(response, query, context_chunks):
    """
    Evaluate the quality of the generated response using Groq
    """
    try:
        # Join context chunks for evaluation
        context = "\n\n".join(context_chunks)
        evaluation_prompt = f"""Evaluate the following response for its quality and relevance:

Query: {query}

Context: {context}

Response: {response}

Please evaluate the response on the following criteria:
1. Relevance to the query
2. Accuracy based on the context
3. Completeness of the answer
4. Clarity and coherence
5. Use of context information

Provide a score from 1-10 for each criterion and a brief explanation."""

        evaluation = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are an evaluation expert."},
                {"role": "user", "content": evaluation_prompt}
            ],
            max_tokens=300,
            temperature=0.3
        )

        return evaluation.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error in evaluate_response_quality: {str(e)}")
        return "Evaluation failed due to an error."

In [93]:
def enhanced_rag_pipeline(index, user_query, embed_model='sentence-transformers/LaBSE',
                         top_k=20, min_score=0.4, max_tokens=3000):
    """
    Enhanced RAG pipeline with improved context handling and response generation using Groq
    """
    try:
        # 1. Embed and retrieve
        model = SentenceTransformer(embed_model)
        query_embedding = model.encode(user_query)

        results = index.query(
            vector=query_embedding.tolist(),
            top_k=top_k,
            include_metadata=True
        )

        # 2. Filter and process results
        matches = [m for m in results.matches if m.score >= min_score]

        # 3. Extract and rank context chunks
        context_chunks = [m.metadata['text'] for m in matches]
        ranked_chunks = rank_context_chunks(context_chunks, user_query)

        # 4. Remove redundant chunks
        unique_chunks = filter_redundant_chunks([chunk for chunk, _ in ranked_chunks])

        # 5. Summarize if needed
        if len(unique_chunks) > 10:
            summarized_chunks = summarize_context(unique_chunks)
        else:
            summarized_chunks = unique_chunks

        # 6. Build final context
        context = "\n\n".join(summarized_chunks)

        # 7. Build enhanced prompt
        prompt = build_enhanced_prompt(context, user_query)

        # 8. Generate response with improved parameters
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=512,
            temperature=0.3,
        )

        response_text = response.choices[0].message.content.strip()

        # Evaluate response quality
        evaluation = evaluate_response_quality(response_text, user_query, summarized_chunks)
        print(f"\nResponse Evaluation:\n{evaluation}")

        return response_text

    except Exception as e:
        print(f"Error in enhanced_rag_pipeline: {str(e)}")
        return "Sorry, I encountered an error while processing your request."


In [94]:
def main():
    try:

        # Initialize Groq client
        groq_api_key = os.getenv("GROQ_API_KEY")
        if not groq_api_key:
            raise ValueError("GROQ_API_KEY environment variable is not set")

        global client
        client = Groq(api_key=groq_api_key)
        # Step 1: Collect multilingual data
        df = collect_multilingual_data()

        # Step 2: Preprocess text
        df = preprocess_text(df)

        # Step 3: Apply enhanced chunking
        df = apply_enhanced_chunking(df)

        # Step 4: Generate multilingual embeddings
        embeddings_dict = generate_multilingual_embeddings(df)

        # Step 5: Setup Pinecone
        index = setup_pinecone()

        # Step 6: Store enhanced embeddings
        store_enhanced_embeddings(index, df, embeddings_dict)

        # Step 7: Test enhanced RAG pipeline
        test_queries = [
            "What are the health benefits of Pakistani cuisine?",
            "پاکستانی کھانوں کے صحت کے فوائد کیا ہیں؟",
            "How can traditional Pakistani food be part of a balanced diet?",
            "روایتی پاکستانی کھانوں کو متوازن غذا کا حصہ کیسے بنایا جا سکتا ہے؟"
        ]

        for query in test_queries:
            print(f"\nQuery: {query}")
            response = enhanced_rag_pipeline(index, query)
            print(f"Response: {response}")


        print("\nAll steps completed successfully!")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


=== Step 1: Collecting Multilingual Data ===



Data Collection Summary:
Total documents: 13
English documents: 11
Urdu documents: 2
Wikipedia sources: 9
Custom sources: 4

=== Step 2: Preprocessing Text ===
Sample of cleaned text (English):
Nutrition is the biochemical and physiological process by which an organism uses food and water to support its life. The intake of these substances provides organisms with nutrients divided into macro

Sample of cleaned text (Urdu):
پاکستانی کھانوں میں استعمال ہونے والے مصالحے صحت کے لیے بہت فائدہ مند ہیں ہلدی میں اینٹی انفلیمیٹری خصوصیات ہیں، زیرہ ہاضمہ کو بہتر بناتا ہے، دھنیا اینٹی آکسیڈنٹس سے بھرپور ہے، الائچی سانس کی بیماریوں

=== Step 3: Applying Enhanced Chunking Strategies ===

Chunking Statistics:
fixed_chunks:
  Total chunks: 41
  Average chunks per document: 3.15
sentence_chunks:
  Total chunks: 40
  Average chunks per document: 3.08
semantic_chunks:
  Total chunks: 13
  Average chunks per document: 1.00

=== Step 4: Generating Multilingual Embeddings ===

Processing fixed_chunks...
T

Generating embeddings with sentence-transformers/LaBSE: 100%|██████████| 2/2 [00:14<00:00,  7.32s/it]


Successfully generated and saved embeddings for fixed_chunks_LaBSE

Processing sentence_chunks...
Total chunks to process: 40

Generating embeddings with sentence-transformers/LaBSE
Loading model: sentence-transformers/LaBSE
Model sentence-transformers/LaBSE generates 768-dimensional embeddings


Generating embeddings with sentence-transformers/LaBSE: 100%|██████████| 2/2 [00:13<00:00,  6.94s/it]


Successfully generated and saved embeddings for sentence_chunks_LaBSE

Processing semantic_chunks...
Total chunks to process: 13

Generating embeddings with sentence-transformers/LaBSE
Loading model: sentence-transformers/LaBSE
Model sentence-transformers/LaBSE generates 768-dimensional embeddings


Generating embeddings with sentence-transformers/LaBSE: 100%|██████████| 1/1 [00:13<00:00, 13.17s/it]


Successfully generated and saved embeddings for semantic_chunks_LaBSE

=== Step 5: Setting up Pinecone ===
Using existing index: urdu-eng-rag
Pinecone setup completed successfully!

=== Step 6: Storing Enhanced Embeddings ===

Processing fixed_chunks...
Storing embeddings for fixed_chunks_LaBSE
Successfully stored batch 1
Successfully stored all embeddings for fixed_chunks_LaBSE

Processing sentence_chunks...
Storing embeddings for sentence_chunks_LaBSE
Successfully stored batch 1
Successfully stored all embeddings for sentence_chunks_LaBSE

Processing semantic_chunks...
Storing embeddings for semantic_chunks_LaBSE
Successfully stored batch 1
Successfully stored all embeddings for semantic_chunks_LaBSE
All embeddings stored successfully!

Query: What are the health benefits of Pakistani cuisine?

Response Evaluation:
Here's the evaluation of the response based on the given criteria:

1. **Relevance to the query: 9**
The response is highly relevant to the query as it directly addresses 